In [1]:
a = [1,2,3,4,5,6,7,8]
b = [1,3,5,7,9,11]

for elem in a and elem not in b:
    print(elem)

NameError: name 'elem' is not defined

In [12]:
a.append(99999)

In [13]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 99999, 99999, 99999, 99999, 99999]

In [27]:
import pandas as pd

data = {
    "title": [],
    "extension": [],
    "imgURL": [],
    "nChar": [],
    "nWords": [],
    "nImg": [],
    "nLinks": [],
    "text": []
    }

frame = pd.DataFrame(data)

In [28]:
frame.title = ["phil", "something", "another", "asshole", "foobar", "fizzbuzz"]

In [30]:
frame.merge(pd.DataFrame({"title": ["new", "newer", "newest"]}), on="title", how = "outer")

,extension,imgURL,nChar,nImg,nLinks,nWords,text,title
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phil
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,something
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,another
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,asshole
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,foobar
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fizzbuzz
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,newer
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,newest


In [34]:
newData = pd.DataFrame({
			"extension": [90],
			"title": ["phil"],
			"imgURL": [12],
		    "nChar": [456],
		    "nWords": [444],
		    "nImg": [2],
		    "nLinks": [3],
		    "text": [1]
		})

frame.merge(newData, on = "title", how="right")

,extension_x,imgURL_x,nChar_x,nImg_x,nLinks_x,nWords_x,text_x,title,extension_y,imgURL_y,nChar_y,nImg_y,nLinks_y,nWords_y,text_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phil,90,12,456,2,3,444,1


In [21]:
ass()[1]

2